# We want to propose a measure to calculate game success based on BGG data

We first import all necessary tables with SQL query

In [338]:
import pandas as pd
import numpy as np
import sql_functions as sf
import Capstone_functions as cp
from IPython.display import clear_output

engine = sf.get_engine()
 
schema = "bgg_data"

In [339]:
sql = f'''Select * from {schema}.marketplace_listings
''' # we calculate the mean price of the boardgame with condition = new

In [340]:
df_price_new = sf.get_dataframe(sql) 

In [341]:
df_price_new

,id,listdate,price,currency,condition,conv_currency,price_in_dollars
0,98443,2018-08-27,50.00,GBP,verygood,1.21,60.5000
1,98443,2019-03-25,115.00,EUR,likenew,1.03,118.4500
2,98443,2019-08-25,110.00,EUR,likenew,1.03,113.3000
3,98443,2020-03-06,150.00,USD,verygood,1.00,150.0000
4,98443,2020-12-24,60.00,EUR,verygood,1.03,61.8000
...,...,...,...,...,...,...,...
241552,298166,2021-08-07,10.00,USD,new,1.00,10.0000
241553,298173,2022-01-30,5.00,EUR,verygood,1.03,5.1500
241554,298175,2020-08-08,34.00,EUR,new,1.03,35.0200
241555,298175,2020-11-03,33.99,EUR,new,1.03,35.0097


In [342]:
df_price_new = df_price_new.query('condition == "new" & price_in_dollars < 150')

In [343]:
df_price_new = df_price_new[['id','price_in_dollars']].groupby('id').median()

In [344]:
df_price_new.reset_index(inplace=True)

In [345]:
df_price_new

,id,price_in_dollars
0,1,87.55000
1,3,102.48500
2,5,45.70110
3,7,50.47000
4,8,36.02425
...,...,...
31377,364780,3.42500
31378,364787,36.03970
31379,364810,29.00000
31380,365073,11.33000


In [346]:
sql = f'select * from {schema}.unfiltered_main_stats_cleaned'

In [347]:
df_main = sf.get_dataframe(sql) 

In [348]:
df_main

,id,yearpublished,min_players,max_players,playtime,min_playtime,max_playtime,min_age,average,user_rated,num_owned,trading,wanting,wishing,numcomments,numweights,averageweight
0,98401,2010.0,2.0,4.0,30.0,30.0,30.0,10,7.08333,3,15,0,1,3,2,2,1.5
1,98402,2011.0,2.0,5.0,30.0,30.0,30.0,8,6.34615,26,20,0,7,13,9,5,2.2
2,98406,2011.0,2.0,4.0,30.0,30.0,30.0,10,6.73333,9,16,3,0,0,3,1,1.0
3,98416,2007.0,2.0,4.0,50.0,50.0,50.0,8,6.80000,2,16,2,1,0,2,1,3.0
4,98417,2009.0,2.0,2.0,60.0,60.0,60.0,12,0.00000,0,10,1,2,2,0,1,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137142,298189,2020.0,3.0,7.0,120.0,30.0,120.0,7,0.00000,0,0,0,0,0,0,0,NaN
137143,298190,2020.0,3.0,6.0,30.0,30.0,30.0,12,6.66667,3,47,2,1,5,4,0,NaN
137144,298191,2022.0,2.0,5.0,80.0,30.0,80.0,8,0.00000,0,5,0,0,0,0,0,NaN
137145,298193,2020.0,2.0,6.0,20.0,10.0,20.0,10,7.38077,26,61,1,0,7,7,2,1.5


In [349]:
df_merged = pd.merge(df_main,df_price_new,on='id')

In [350]:
df_merged

,id,yearpublished,min_players,max_players,playtime,min_playtime,max_playtime,min_age,average,user_rated,num_owned,trading,wanting,wishing,numcomments,numweights,averageweight,price_in_dollars
0,98479,2011.0,2.0,5.0,60.0,60.0,60.0,10,8.06250,8,61,11,1,5,12,0,NaN,2.57500
1,98527,2017.0,1.0,4.0,180.0,60.0,180.0,13,7.57245,1719,3577,73,288,1585,628,81,3.0988,95.00000
2,98533,2008.0,2.0,12.0,30.0,30.0,30.0,12,7.71429,7,41,1,2,6,3,1,3.0000,21.72925
3,98552,2010.0,2.0,4.0,60.0,60.0,60.0,13,6.99375,40,355,11,6,17,16,1,1.0000,9.63500
4,98590,2011.0,2.0,2.0,30.0,30.0,30.0,13,7.38936,47,393,8,14,17,19,2,3.0000,13.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28724,298151,2020.0,2.0,4.0,20.0,20.0,20.0,6,5.42500,10,28,3,1,7,1,0,NaN,9.24425
28725,298154,2020.0,2.0,6.0,240.0,120.0,240.0,14,7.49775,111,330,7,39,101,36,8,4.3750,75.00000
28726,298166,2020.0,2.0,6.0,5.0,5.0,5.0,8,6.57194,134,379,3,3,23,29,8,1.1250,10.00000
28727,298175,2020.0,2.0,6.0,180.0,60.0,180.0,13,7.37500,8,80,1,6,28,2,0,NaN,35.01485


In [351]:
df_merged_est_revenue = df_merged[['id','num_owned','price_in_dollars']]

In [352]:
df_merged_est_revenue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28729 entries, 0 to 28728
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                28729 non-null  int64  
 1   num_owned         28729 non-null  int64  
 2   price_in_dollars  28729 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 897.8 KB


We create a calculate field with num_owned x average new price

In [353]:
df_merged_est_revenue['revenue'] = df_merged_est_revenue['num_owned'] * df_merged_est_revenue['price_in_dollars']

/var/folders/rb/p48bcgy90nn3g9bz4kw5bh4w0000gn/T/ipykernel_19071/3826108597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_est_revenue['revenue'] = df_merged_est_revenue['num_owned'] * df_merged_est_revenue['price_in_dollars']


In [354]:
df_merged_est_revenue.sort_values('revenue',ascending=False)

,id,num_owned,price_in_dollars,revenue
27924,174430,82352,108.90000,8.968133e+06
19948,169786,92395,72.10000,6.661680e+06
4611,30549,175618,37.74950,6.629492e+06
14853,167791,109606,58.83485,6.448653e+06
23870,13,175387,36.68485,6.434046e+06
...,...,...,...,...
22689,308352,0,44.95000,0.000000e+00
16429,252771,0,10.78000,0.000000e+00
22605,240819,0,25.00000,0.000000e+00
16681,33284,0,25.00000,0.000000e+00


In [355]:
mask = df_merged_est_revenue['id'] == 822
df_merged_est_revenue.loc[mask,:]

,id,num_owned,price_in_dollars,revenue
5678,822,167688,26.00235,4.360282e+06


In [356]:
df_merged_est_revenue.drop_duplicates(inplace=True)

/var/folders/rb/p48bcgy90nn3g9bz4kw5bh4w0000gn/T/ipykernel_19071/991260501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_est_revenue.drop_duplicates(inplace=True)


In [357]:
sf.build_table(engine=engine,table_name=f"clean_revenue",dataframe=df_merged_est_revenue,schema=schema)

The clean_revenue table was imported successfully.


## Work on expansions

In [358]:
# 822 - Carcassonne



,id,expansion_id,expansion_name
14939,822,167903,20 Jahre Darmstadt Spielt
14940,822,318136,Apothecaries (fan expansion for Carcassonne)
14941,822,248789,Apothecaries and Tithes (fan expansion for Car...
14942,822,362915,Archery Tournament (fan expansion for Carcasso...
14943,822,318408,Die Bettler (fan-erweiterung für Carcassonne)
...,...,...,...
15101,822,160096,Каркассон: Дворяне и Башни
15102,822,211659,Каркассон: Наука и магия
15103,822,160095,Каркассон: Предместья и обитатели
15104,822,215144,Каркассон: Солове́й-Разбо́йник & Водяной
